In [1]:
import csv
import os
from batid.services.guess_bdg_new import Guesser, GeocodeNameHandler


os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"


guesses_path = "guesses.json"


In [2]:
def row_to_input(row):
    
    return {
        "ext_id": row['\ufeff"Id Unique"'],
        "name": row["Nom"],
        "lat": float(row["Latitude"]),
        "lng": float(row["Longitude"])
    }

In [4]:
file_path = "places.csv"

inputs = []

ext_ids = set()

with open(file_path, "r") as f:
    reader = csv.DictReader(f, delimiter=";")
    
    for row in reader:
        
        # It seems there are some duplicates in the original file
        if row['\ufeff"Id Unique"'] in ext_ids:
            continue
        
        ext_ids.add(row['\ufeff"Id Unique"'])
        
        input = row_to_input(row)
        inputs.append(input)

 
print(f"There are {len(ext_ids)} unique id in the original file")

guesser = Guesser()
guesser.create_work_file(inputs, guesses_path)

There are 39500 unique id in the original file
- saving work file
- work file saved


In [2]:
guesser = Guesser()
guesser.handlers = [GeocodeNameHandler()]
guesser.guess_work_file(guesses_path, 100)

- loading work file
- converting guesses to batches
- converted 39500 guesses to 395 batches
Batch 1/395
Batch did not change
Batch 2/395
Batch did not change
Batch 3/395
Batch did not change
Batch 4/395
Batch did not change
Batch 5/395
Batch did not change
Batch 6/395
Batch did not change
Batch 7/395
Batch did not change
Batch 8/395
Batch did not change
Batch 9/395
Batch did not change
Batch 10/395
Batch did not change
Batch 11/395
Batch did not change
Batch 12/395
Batch did not change
Batch 13/395
Batch did not change
Batch 14/395
Batch did not change
Batch 15/395
Batch did not change
Batch 16/395
Batch did not change
Batch 17/395
Batch did not change
Batch 18/395
Batch did not change
Batch 19/395
Batch did not change
Batch 20/395
Batch did not change
Batch 21/395
Batch did not change
Batch 22/395
Batch did not change
Batch 23/395
Batch did not change
Batch 24/395
Batch did not change
Batch 25/395
Batch did not change
Batch 26/395
Batch did not change
Batch 27/395
Batch did not chang

Batch changed
- saving work file
- work file saved
Batch 260/395
Batch changed
- saving work file
- work file saved
Batch 261/395
Batch changed
- saving work file
- work file saved
Batch 262/395
Batch changed
- saving work file
- work file saved
Batch 263/395
Batch changed
- saving work file
- work file saved
Batch 264/395
Batch changed
- saving work file
- work file saved
Batch 265/395
Batch changed
- saving work file
- work file saved
Batch 266/395
Batch changed
- saving work file
- work file saved
Batch 267/395
Batch changed
- saving work file
- work file saved
Batch 268/395
Batch changed
- saving work file
- work file saved
Batch 269/395
Batch changed
- saving work file
- work file saved
Batch 270/395
Batch changed
- saving work file
- work file saved
Batch 271/395
Batch changed
- saving work file
- work file saved
Batch 272/395
Batch changed
- saving work file
- work file saved
Batch 273/395
Batch changed
- saving work file
- work file saved
Batch 274/395
Batch changed
- saving wo

Batch changed
- saving work file
- work file saved
Batch 387/395
Batch changed
- saving work file
- work file saved
Batch 388/395
Batch changed
- saving work file
- work file saved
Batch 389/395
Batch changed
- saving work file
- work file saved
Batch 390/395
Batch changed
- saving work file
- work file saved
Batch 391/395
Batch changed
- saving work file
- work file saved
Batch 392/395
Batch changed
- saving work file
- work file saved
Batch 393/395
Batch changed
- saving work file
- work file saved
Batch 394/395
Batch changed
- saving work file
- work file saved
Batch 395/395
Batch changed
- saving work file
- work file saved


In [2]:
from IPython.display import display, HTML

guesser = Guesser()
guesser.load_work_file(guesses_path)
guesser.report()

sample = guesser.matched_sample("found_name_in_osm", 20, ["input_ext_id", "matches", "match_reason", "input_name", "input_lat", "input_lng"],)

display(HTML(sample.to_html()))

- loading work file
-- Report --
Number of rows: 39500
Number of match: 19621 (49.67%)

-- finished_steps --
Rows with finished_steps closest_from_point: 0 (0.00%)
Rows with finished_steps geocode_address: 0 (0.00%)
Rows with finished_steps geocode_name: 39500 (100.00%)
Rows with empty finished_steps: 0 (0.00%)

-- match_reasons : absolute --
match_reason
found_name_in_osm    19621
Name: count, dtype: int64

-- match_reasons : % --
match_reason
found_name_in_osm    49.673418
Name: count, dtype: float64

-- Inputs --


,input_ext_id,matches,match_reason,input_name,input_lat,input_lng
11222,c07077ea-339c-50a1-a109-1451d48f9bde,[D2CZJ9Y9QS4W],found_name_in_osm,Maison Saint-Jean,48.074439,7.357466
403,21b2041d-14f5-4f63-ba9d-b0c5713074b2,[6VQR68AA73AV],found_name_in_osm,Médiathèque Départementale Des Hautes-Pyrénées,43.219596,0.068133
550,2d328182-5ab5-5888-969c-c4a8891e6f73,[6ZAE6Q5154YQ],found_name_in_osm,Église Saint-Antoine-l'Ermite,45.402565,2.065242
7622,846091a8-703f-461d-ba13-c30f7d7c40bc,[WNA35AJTGK5R],found_name_in_osm,Develay,45.990559,4.718717
34950,a938eea6-397e-5510-b06a-662708b29ae6,[NG31P8X18XDQ],found_name_in_osm,Maison de la Grande Vigne,48.461487,-2.032800
2672,e205e6d1-d4fe-5c76-8db3-86a5df8ba507,[AHWJN5SXD281],found_name_in_osm,Église catholique Saint-Michel,48.017498,7.328407
15552,3129e194-b68f-563d-8bed-0451cd699272,[1AYH2QVBDZS6],found_name_in_osm,Ancien château de Gayette,46.335263,3.447705
33670,38f6df76-8016-5a89-8d2f-cb565b3cbe8e,[SA6Y3K1GRSWZ],found_name_in_osm,Bibliothèque De Saint-Sauveur,48.447789,-4.001535
24984,513027b8-2c5b-5523-8b23-41338ec7bd62,[1TZQ5JEH488Y],found_name_in_osm,Chapelle d'Aiguevives,44.484819,0.603745
8640,dd190a2d-e50c-5f50-b45c-1d38d0c79d1c,[SQVSZ72QNBB3],found_name_in_osm,Église Saint-Amand,44.979819,2.935327
